# Building a Transformer Based Model from Scratch

Video link: https://www.youtube.com/watch?v=kCc8FmEb1nY

In [16]:
# set the dataset you want to train on

dataset_dir = '../data/tiny-shakespeare.txt'

In [17]:
# read the dataset and inspection

with open(dataset_dir, 'r', encoding='utf-8') as f:
    text = f.read()

In [19]:
print('length of the dataset: ', len(text))

length of the dataset:  1115394


In [20]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [21]:
# building the vocabulary -- all the unique characters that occur in the text

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# tokenization is only needed for words-models. This one is based only on characters, so char=token

In [ ]:
# create a mapping from characters to integers -- models can only work with numbers

stoi = { ch:i for i,ch in enumerate(chars) }    # string to index
itos = { i:ch for i,ch in enumerate(chars) }    # index to string
encode = lambda s: [stoi[c] for c in s] # encoder: take a str, output a list of ints
decode = lambda l: ''.join([itos[c] for c in l]) # decoder: take a list of ints, outputs a str

print(encode('pepelin'))
print(decode(encode('pepelin')))


[54, 43, 54, 43, 50, 47, 52]
pepelin


minuto 10:30